In [30]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import pandas as pd
import numpy as np

def scrape_indicator():
    indicator_url = "https://www.tradingview.com/markets/world-economy/indicators/"
    prefix = "https://www.tradingview.com/symbols/ECONOMICS-"
    country_suffix = ["US", "CN", "EU", "JP", "DE", "GB", "FR", "RU", "CA", "IT", "AU"]
    indicator_suffix = ['GDP', 'POP', 'GDPYY', 'INTR', 'IRYY', 'UR', 'CAG', 'GDG']
    
    chrome_options = Options()
    chrome_options.binary_location = r"C:\Program Files\Google\Chrome\Application\chrome.exe"
    driver = webdriver.Chrome(options=chrome_options)
    driver.get(indicator_url)
    
    try:
        # wait for the web content to load
        target = WebDriverWait(driver, 10).until(
            EC.visibility_of_element_located((By.XPATH, '//a[contains(@href,"ECONOMICS-USGDP")]'))
        )
        
        # init value dataframe
        value_df = pd.DataFrame(np.zeros((len(indicator_suffix), len(country_suffix) + 1)), columns=['indicator'] + country_suffix)
        value_df['indicator'] = pd.Series(indicator_suffix)
        
        for country in country_suffix:
            for indicator in indicator_suffix:
                indicator_path = f'//a[contains(@href,"ECONOMICS-{country + indicator}")]'
                indicator_value = driver.find_element(By.XPATH, indicator_path).find_element(By.CSS_SELECTOR, "span").text
                value_df.loc[indicator_suffix.index(indicator), country] = indicator_value
        
        return value_df
    
    finally:
        # close window
        driver.quit()

# 执行爬取并打印结果
indicator = scrape_indicator()
indicator

,indicator,US,CN,EU,JP,DE,GB,FR,RU,CA,IT,AU
0,GDP,25.44T,17.963T,14.136T,4.232T,4.082T,3.089T,2.779T,2.24T,2.138T,2.05T,1.693T
1,POP,334.229M,1.41B,345.41M,124.95M,84.4M,67.79M,68.14M,145.6M,39.29M,58.98M,26.27M
2,GDPYY,3.10%,5.20%,0.10%,1.00%,−0.20%,−0.20%,0.70%,5.50%,0.47%,0.50%,2.10%
3,INTR,5.50%,3.45%,4.50%,−0.10%,4.50%,5.25%,4.25%,16.00%,5.00%,4.25%,4.35%
4,IRYY,3.10%,−0.80%,2.90%,2.60%,2.90%,4.00%,3.10%,7.40%,3.40%,0.76%,4.10%
5,UR,3.70%,5.10%,6.40%,2.40%,5.80%,3.80%,7.50%,3.00%,5.70%,7.20%,4.10%
6,CAG,−3.70%,2.20%,−0.60%,1.80%,4.20%,−5.60%,−2.00%,10.30%,−0.40%,−1.50%,0.00%
7,GDG,129.00%,77.10%,90.90%,263.90%,66.10%,97.10%,111.80%,17.20%,107.00%,141.70%,22.30%


In [31]:
import datetime

today = datetime.datetime.now().date()
str(today)

'2024-02-16'